In [ ]:
import os, h5py, numpy as np
import illustris_python as il
import yt
import yt.units as ytu
from emis import emissivity
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as c
from plot_utils import quiver_from_components

In [ ]:
with h5py.File('mw_tng_sample_catalog.hdf5') as f:
    mw_id = np.array(f['SubfindID'])

In [ ]:
#outpath = './cutouts/TNG50_snap099_fof000020_gas_sphere_500kpc.hdf5'
subid = mw_id[5]
outpath = f'/home/cj535/palmer_scratch/TNG50_cutouts/MW_sample/TNG50_snap099_subid{subid:06d}_gas_sphere_500kpc.hdf5'
idx_path = outpath.replace(".hdf5", ".ytindex")
ds = yt.load(outpath, index_filename=idx_path,smoothing_factor=2)
#ds.add_deposited_particle_field(("gas","Halpha_power"), "sum")

def _Ha_cloudy(field, data):
    nH = data["gas","H_number_density"].in_units("1/cm**3").astype("float64")
    T  = data["gas","temperature"].in_units("K").astype("float64")
    Z  = data["gas","metallicity"].to("dimensionless").astype("float64")                
    # your helper returns log10 emissivity
    log_eps = emissivity("H-alpha", nH.value, Z.value, T.value, redshift=float(data.ds.current_redshift))
    eps = np.power(10.0, log_eps, dtype=np.float64) * data['gas','H_fraction'] * ytu.erg/ytu.s/ytu.cm**3
    return eps
ds.add_field(
    name=("gas", "Halpha_emissivity"),
    function=_Ha_cloudy,
    sampling_type="particle",
    units="erg/s/cm**3"
)
def _Ha_brightness(field, data):
    return data[("gas","Halpha_emissivity")] / (4*np.pi) / ytu.sr
ds.add_field(
    name=("gas", "Halpha_brightness"),
    function=_Ha_brightness,
    sampling_type="particle",
    units="erg/s/cm**3/arcsec**2"
)

def _Ha_pow(field, data):
    return data[("gas","Halpha_emissivity")] * data[("gas","cell_volume")]
ds.add_field(
    name=("gas", "Halpha_power"),
    function=_Ha_pow,
    sampling_type="particle",
    units="erg/s"
)
center_comov = 0.5 * ds.domain_width          # code center (comoving)
center_prop  = (center_comov.to("kpc") * ds.scale_factor)
sp_small = ds.sphere(center_prop, (100.0, "kpc"))
bv = sp_small.quantities.bulk_velocity()
'''
def _dv_los(field,data):
    v = data[("gas","velocity_los")]
    ax = data.get_field_parameter("axis")
    if yt.funcs.is_sequence(ax):
        # Make sure this is a unit vector
        ax /= np.sqrt(np.dot(ax, ax))
        v0 = bv[0]*ax[0] + bv[1]*ax[1] + bv[2]*ax[2]
        v0 = v0
    elif ax in [0,1,2]:
        v0 = bv[ax]
    else:
        raise NeedsParameter(["axis"])
    return v - v0
ds.add_field(
    name=("gas", "dv_los"),
    function=_dv_los,
    units="km/s",
    sampling_type="particle"
)
'''

def _mask(lo, hi):
    def _f(field, data):      
        dv = data[("gas","velocity_los")].to_value("km/s")
        return ((dv >= lo) & (dv < hi)).astype("float64")  # 1 inside, 0 outside
    ds.add_field(
        name=("gas", f"mask_{lo:.0f}_{hi:.0f}"),
        function=_f,
        units="",
        sampling_type="particle"
    )
    def _g(field, data):
        return data[("gas","Halpha_emissivity")] * data[("gas",f"mask_{lo:.0f}_{hi:.0f}")]
    ds.add_field(
        name=("gas", f"Halpha_emissivity_{lo:.0f}_{hi:.0f}"),
        function=_g,
        units="erg/s/cm**3",
        sampling_type="particle"
    )
    def _h(field, data):
        return data[("gas","Halpha_brightness")] * data[("gas",f"mask_{lo:.0f}_{hi:.0f}")]
    ds.add_field(
        name=("gas", f"Halpha_brightness_{lo:.0f}_{hi:.0f}"),
        function=_h,
        units="erg/s/cm**3/arcsec**2",
        sampling_type="particle"
    )

    return _f, _g

_mask(-300, -100); _mask(-100, 100); _mask(100, 300)
   
ds.derived_field_list

In [ ]:
print(ds.field_info[('gas', 'relative_velocity_x')].get_source())

In [ ]:
width_proper = (500.0, "kpc")                 # proper
center_comov = 0.5 * ds.domain_width          # code center (comoving)
center_prop  = (center_comov.to("kpc") * ds.scale_factor)

theta = np.deg2rad(15.0)
w_hat = np.array([np.sin(theta), 0.0, np.cos(theta)])  # (nx, ny, nz)
v_hat = np.array([0.0, 1.0, 0.0])  # +y points "up" on the image
u_hat = np.cross(v_hat,w_hat)

#unit_vector /= np.sqrt(np.dot(unit_vector,unit_vector)) #make sure it's unit
def _vel_u(field,data):
    ux,uy,uz = u_hat[0],u_hat[1],u_hat[2]
    vx = data['gas','relative_velocity_x']
    vy = data['gas','relative_velocity_y']
    vz = data['gas','relative_velocity_z']
    return vx*ux + vy*uy + vz*uz
ds.add_field(
    name=("gas", f"velocity_u"),
    function=_vel_u,
    units="km/s",
    sampling_type="particle"
)
def _momentum_density_u(field,data):
    return data['gas','velocity_u']*data['gas','density']
ds.add_field(name=('gas','momentum_density_u'),function=_momentum_density_u,units="km*g/cm**3/s",sampling_type="particle")

def _vel_v(field,data):
    ux,uy,uz = v_hat[0],v_hat[1],v_hat[2]
    vx = data['gas','relative_velocity_x']
    vy = data['gas','relative_velocity_y']
    vz = data['gas','relative_velocity_z']
    return vx*ux + vy*uy + vz*uz
ds.add_field(
    name=("gas", f"velocity_v"),
    function=_vel_v,
    units="km/s",
    sampling_type="particle"
)
def _momentum_density_v(field,data):
    return data['gas','velocity_v']*data['gas','density']
ds.add_field(name=('gas','momentum_density_v'),function=_momentum_density_v,units="km*g/cm**3/s",sampling_type="particle")

def _vel_w(field,data):
    ux,uy,uz = w_hat[0],w_hat[1],w_hat[2]
    vx = data['gas','relative_velocity_x']
    vy = data['gas','relative_velocity_y']
    vz = data['gas','relative_velocity_z']
    return vx*ux + vy*uy + vz*uz
ds.add_field(
    name=("gas", f"velocity_w"),
    function=_vel_w,
    units="km/s",
    sampling_type="particle"
)
def _momentum_density_w(field,data):
    return data['gas','velocity_w']*data['gas','density']
ds.add_field(name=('gas','momentum_density_w'),function=_momentum_density_w,units="km*g/cm**3/s",sampling_type="particle")



In [ ]:
width_proper = (500.0, "kpc")                 # proper
center_comov = 0.5 * ds.domain_width          # code center (comoving)
center_prop  = (center_comov.to("kpc") * ds.scale_factor)
sp_small = ds.sphere(center_prop, (50.0, "kpc"))
bv = sp_small.quantities.bulk_velocity()
sp = ds.sphere(center_prop, (500.0, "kpc"))
sp.set_field_parameter("bulk_velocity", bv)
resolution = (256,256)

field = ("gas","velocity_u")
p = yt.ProjectionPlot(
    ds, w_hat, field,
    center=center_prop, width=width_proper,
    north_vector=v_hat,
    weight_field=("gas","density"),
    buff_size=resolution,
    data_source=sp
)
img = p.frb[field]
p.set_log(field, False)
vel_u = img.to_value(img.units)
p.show()

field = ("gas","velocity_v")
p = yt.ProjectionPlot(
    ds, w_hat, field,
    center=center_prop, width=width_proper,
    north_vector=v_hat,
    weight_field=("gas","density"),
    buff_size=resolution,
    data_source=sp
)
img = p.frb[field]
p.set_log(field, False)
vel_v = img.to_value(img.units)
p.show()

field = ("gas","velocity_w")
p = yt.ProjectionPlot(
    ds, w_hat, field,
    center=center_prop, width=width_proper,
    north_vector=v_hat,
    weight_field=("gas","density"),
    buff_size=resolution,
    data_source=sp
)
img = p.frb[field]
vel_w = img.to_value(img.units)
p.set_log(field, False)
p.show()

field_list = [("gas","Halpha_brightness"),
              ("gas","Halpha_brightness_-300_-100"),
              ("gas","Halpha_brightness_-100_100"),
              ("gas","Halpha_brightness_100_300")]
p = yt.ProjectionPlot(
    ds, w_hat, field_list,
    center=center_prop, width=width_proper,
    north_vector=v_hat,
    method="integrate",
    buff_size=resolution,
    data_source=sp
)
HalphaL = {}
range_name = ['bolo','-300_-100','-100_100','100_300']
for i, field in enumerate(field_list):
    img = p.frb[field]
    HalphaL[range_name[i]] = img.to_value(img.units)
    p.set_zlim(field, zmin=5e-21, zmax=1e-18)

p.show()

In [ ]:
field = ("gas","momentum_density_u")
p = yt.ProjectionPlot(
    ds, w_hat, field,
    center=center_prop, width=width_proper,
    north_vector=v_hat,
    buff_size=resolution,
    data_source=sp
)
img = p.frb[field]
momentum_u = img.to_value(img.units)
p.set_log(field, False)
p.set_zlim(field, zmin=-0.1, zmax=0.1)
p.show()

field = ("gas","momentum_density_v")
p = yt.ProjectionPlot(
    ds, w_hat, field,
    center=center_prop, width=width_proper,
    north_vector=v_hat,
    buff_size=resolution,
    data_source=sp
)
img = p.frb[field]
momentum_v = img.to_value(img.units)
p.set_log(field, False)
p.set_zlim(field, zmin=-0.1, zmax=0.1)
p.show()

field = ("gas","momentum_density_w")
p = yt.ProjectionPlot(
    ds, w_hat, field,
    center=center_prop, width=width_proper,
    north_vector=v_hat,
    buff_size=resolution,
    data_source=sp
)
img = p.frb[field]
momentum_w = img.to_value(img.units)
p.set_log(field, False)
p.set_zlim(field, zmin=-0.1, zmax=0.1)
p.show()

In [ ]:
ny, nx = vel_v.shape
x = np.arange(nx)
y = np.arange(ny)
X, Y = np.meshgrid(x, y)
R = np.sqrt((X-resolution[0]/2)**2 + (Y-resolution[1]/2)**2)
pc_per_pix = 500/resolution[0]
mask = np.array(R > 30/pc_per_pix,dtype=int)


step=4
# subsample
sl = (slice(None, None, step), slice(None, None, step))
Xs, Ys = X[sl], Y[sl]
Vx, Vy, Vz = (mask*vel_u)[sl], (mask*vel_v)[sl], (mask*vel_w)[sl]
px, py, pz = (mask*momentum_u)[sl], (mask*momentum_v)[sl], (mask*momentum_w)[sl]

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12,12))

Hmap = np.log10(HalphaL['bolo'])
Hmap = Hmap * mask

ax.imshow(Hmap,origin='lower',cmap='grey',vmin=-21,vmax=-18)
q = ax.quiver(Xs,Ys,Vx,Vy,Vz,cmap='RdBu_r',clim=(-300,300))
#q = ax.quiver(Xs,Ys,px,py,pz,cmap='RdBu_r',clim=(-0.05,0.05))
#cb = plt.colorbar(q, ax=ax, label='speed (|v|)')


In [ ]:
Ha_sb = Ha_np *u.erg/u.s/u.cm**2 / (4*np.pi*u.rad**2)
Ha_sb = Ha_sb.to(u.erg/u.s/u.cm**2/u.arcsec**2)
plt.imshow(np.log10(Ha_sb.value),origin='lower')
plt.colorbar()

In [ ]:
width_proper = (500.0, "kpc")                 # proper
center_comov = 0.5 * ds.domain_width          # code center (comoving)
center_prop  = (center_comov.to("kpc") * ds.scale_factor)

p = yt.ProjectionPlot(ds, "x", ("gas", "Halpha_brightness"),method="integrate",center=center_prop,width=width_proper)
p.set_unit(("gas","Halpha_brightness"), "erg/s/cm**2/arcsec**2")
p.set_zlim(("gas","Halpha_brightness"), vmin=1e-20, vmax=1e-18)
p.show()

In [ ]:
p.set_unit(("gas","Halpha_brightness"), "erg/s/cm**2/arcsec**2")
p.set_zlim(("gas","Halpha_brightness"), zmin=5e-21, zmax=1e-18)
p.show()

In [ ]:
width_proper = (500.0, "kpc")                 # proper
center_comov = 0.5 * ds.domain_width          # code center (comoving)
center_prop  = (center_comov.to("kpc") * ds.scale_factor)
fields = [
    ("gas", "Halpha_emissivity"),
    ("gas", "Halpha_emissivity_-300_-100"),
    ("gas", "Halpha_emissivity_-100_100"),
    ("gas", "Halpha_emissivity_100_300")
]

p = yt.ProjectionPlot(ds, "x", fields,method="integrate",center=center_prop,width=width_proper)
p.show()

In [ ]:
width_proper = (500.0, "kpc")                 # proper
center_comov = 0.5 * ds.domain_width          # code center (comoving)
center_prop  = (center_comov.to("kpc") * ds.scale_factor)
p = yt.ProjectionPlot(ds, "x", ("gas", "Halpha_emissivity_100_300"),method="integrate",center=center_prop,width=width_proper)
p.show()

In [ ]:
width_proper = (500.0, "kpc")                 # proper
center_comov = 0.5 * ds.domain_width          # code center (comoving)
center_prop  = (center_comov.to("kpc") * ds.scale_factor)

sp = ds.sphere(center_prop, (500.0, "kpc"))
sp.set_field_parameter("bulk_velocity", bv)

field = ('gas', 'velocity_los')
'''[("gas","dv_los"),
         ('gas', 'velocity_los'),
         ('gas', 'velocity_cylindrical_z')]'''

p = yt.SlicePlot(ds, "x", field,center=center_prop,width=width_proper,data_source=sp)
p.set_log(field, False)
p.set_unit(field, "km/s")
vlim = 300  # km/s, set what you like
#p.set_zlim(field, -vlim, vlim)
#p.zoom(2.0)
'''
p.annotate_quiver(
   ("gas", "velocity_x"),
   ("gas", "velocity_y"),
   factor=16,
   color="red",
)

p.annotate_quiver(
   ("gas", "dv_north"),
   ("gas", "dv_east"),
   factor=16,
   color="white",
)
'''
p.show()

In [ ]:
width_proper = (500.0, "kpc")                 # proper
center_comov = 0.5 * ds.domain_width          # code center (comoving)
center_prop  = (center_comov.to("kpc") * ds.scale_factor)

yt.ParticleProjectionPlot(ds,'x',('gas','Halpha_power'),deposition="cic",
                         center=center_prop,width=width_proper)

In [ ]:
#!/usr/bin/env python3
import os, h5py, numpy as np
import illustris_python as il
import yt

# ---------- user params ----------
basepath  = "/gpfs/gibbs/pi/nagai/IllustrisTNG_data/TNG50-1/output"
snapnum   = 99
fof_id    = 200
outdir    = "./cutouts"
fname     = f"TNG50_snap{snapnum:03d}_fof{fof_id:06d}_cutout.hdf5"
include   = ["gas", "dm", "stars", "bh"]  # choose which particle types to save
pad_ckpch = 5.0          # pad the local box by this many ckpc/h on each side
# ---------------------------------

os.makedirs(outdir, exist_ok=True)
outpath = os.path.join(outdir, fname)

# Header for cosmology/units
hdr = il.groupcat.loadHeader(basepath, snapnum)
Time         = float(hdr["Time"])          # scale factor a
HubbleParam  = float(hdr["HubbleParam"])   # h
Omega0       = float(hdr.get("Omega0", 0.3089))
OmegaLambda  = float(hdr.get("OmegaLambda", 0.6911))
BoxSize_full = float(hdr["BoxSize"])       # ckpc/h

# FoF center (ckpc/h) – just for reference if you want it
grp = il.groupcat.loadSingle(basepath, snapnum, haloID=fof_id)
center_ckpch = grp["GroupPos"]  # shape (3,)

# Mapping for types and fields to try (TNG native names)
ptype_map = {
    "gas":   ("PartType0", ["Coordinates","Velocities","Masses",
                            "Density","InternalEnergy",
                            "ElectronAbundance","GFM_Metallicity","GFM_Metals"]),
    "dm":    ("PartType1", ["Coordinates","Velocities","Masses","ParticleIDs"]),
    "stars": ("PartType4", ["Coordinates","Velocities","Masses","GFM_Metallicity","GFM_Metals","StellarFormationTime"]),
    "bh":    ("PartType5", ["Coordinates","Velocities","Masses","BH_Mass","BH_Mdot"]),
}

def load_halo_parttype(ptname):
    """Load all *halo-attached* particles of one type with illustris_python."""
    gname, want = ptype_map[ptname]
    fields = []
    # Only request fields that actually exist for this parttype, or il will raise
    probe = il.snapshot.loadHalo(basepath, snapnum, fof_id, partType=ptname, fields=["Coordinates"])
    # Build a field list by testing on a tiny subfile is expensive; instead we try/except during load
    for f in want:
        try:
            arr = il.snapshot.loadHalo(basepath, snapnum, fof_id, partType=ptname, fields=[f])
            fields.append(f)
        except Exception:
            pass
    # Now load all at once
    data = il.snapshot.loadHalo(basepath, snapnum, fof_id, partType=ptname, fields=fields)
    return gname, data

# 1) Load selected types
loaded = []
for pt in include:
    try:
        gname, data = load_halo_parttype(pt)
        if data["Coordinates"].size == 0:
            continue
        loaded.append((pt, gname, data))
        print(f"Loaded {pt}: {data['Coordinates'].shape[0]:,} particles.")
    except Exception as e:
        print(f"Skip {pt}: {e}")

if not loaded:
    raise RuntimeError("No particles loaded; check inputs.")

# 2) Build a *local* Gadget-like box in ckpc/h around these particles to keep yt fast.
#    We shift positions so that the minimum of each axis is ~pad, and set BoxSize to the span+2*pad.
all_pos = np.vstack([d["Coordinates"] for (_,_,d) in loaded])
mins = all_pos.min(axis=0)
maxs = all_pos.max(axis=0)

lo = mins - pad_ckpch
hi = maxs + pad_ckpch
local_boxsize = (hi - lo)  # vector per axis (ckpc/h)

# 3) Write the mini snapshot
with h5py.File(outpath, "w") as f:
    # Header
    h = f.create_group("Header")
    # counts per type (0..5)
    counts = np.zeros(6, dtype=np.uint32)
    type_index = {"PartType0":0, "PartType1":1, "PartType2":2, "PartType3":3, "PartType4":4, "PartType5":5}
    for (_, gname, data) in loaded:
        counts[type_index[gname]] = data["Coordinates"].shape[0]
    for key, val in [
        ("NumPart_ThisFile", counts),
        ("NumPart_Total",    counts.astype(np.uint32)),
        ("NumPart_Total_HighWord", np.zeros(6, dtype=np.uint32)),
        ("MassTable", np.zeros(6, dtype=np.float64)),   # variable masses
        ("Time", Time),
        ("Redshift", 1.0/Time - 1.0),
        ("BoxSize", float(local_boxsize.max())),        # single BoxSize; use the max axis length
        ("NumFilesPerSnapshot", 1),
        ("Omega0", Omega0),
        ("OmegaLambda", OmegaLambda),
        ("HubbleParam", HubbleParam),
        ("Flag_Entropy", 0),
        ("OriginalBoxSize", BoxSize_full),
        ("CutoutCenter_ckpch", center_ckpch),
        ("CutoutLo_ckpch", lo),
        ("CutoutHi_ckpch", hi),
    ]:
        h.attrs[key] = val

    # Particle groups
    for (pt, gname, data) in loaded:
        g = f.create_group(gname)
        # Shifted coordinates into local box (still in ckpc/h, just re-zeroed)
        coords = data["Coordinates"] - lo[None, :]
        g.create_dataset("Coordinates", data=coords, dtype=np.float32)

        # Velocities (km/s) and Masses (1e10 Msun/h) if present
        if "Velocities" in data:
            g.create_dataset("Velocities", data=data["Velocities"].astype(np.float32))
        if "Masses" in data:
            g.create_dataset("Masses", data=data["Masses"].astype(np.float32))

        # IDs if present
        if "ParticleIDs" in data:
            g.create_dataset("ParticleIDs", data=data["ParticleIDs"].astype(np.uint64))

        # Copy any other present fields verbatim (native units)
        for extra in ["Density","InternalEnergy","ElectronAbundance",
                      "GFM_Metallicity","GFM_Metals","BH_Mass","BH_Mdot",
                      "StellarFormationTime"]:
            if extra in data:
                g.create_dataset(extra, data=data[extra])

print(f"Wrote cutout: {outpath}")

In [ ]:
@yt.derived_field(name=("gas","H_number_density"), units="1/cm**3", sampling_type="cell")
def _nH(field, data):
    rho = data[gas_density]
    X_H = data[gas_metals][:, 0]
    return (rho * X_H) / mp

@yt.derived_field(name=("gas","Halpha_emissivity_cloudy"), units="erg/s/cm**3", sampling_type="cell")
def _Ha_cloudy(field, data):
    nH = data[("gas","H_number_density")]        # 1/cm^3
    T  = data[gas_temperature]                   # K
    Z  = data[gas_Z]                             # mass fraction
    # your helper returns log10 emissivity
    log_eps = emissivity("H-alpha", nH.d, Z.d, T.d, redshift=z)
    return (10.0**log_eps) * (yt.units.erg/yt.units.s/yt.units.cm**3) #from mo further multiplies this by data[gas_metals][:, 0] ???
